In [1]:
import sys

In [2]:
sys.path.insert(1, '../')

In [3]:
from utils.packages import *

/var/local/home/linler17/.conda/envs/masterenv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path_patients = '../data/datasets/readmission_data_synthea'
#prior1 = pd.read_parquet('../data/datasets/prior_table_1')
#prior2 = pd.read_parquet('../data/datasets/prior_table_2')
#priordataset = pd.concat([prior1, prior2])
#data = pd.read_parquet(path_patients)
#dataset = data.merge(priordataset, on='subject_id', how='inner')
dataset = pd.read_parquet(path_patients)

In [6]:
global_params = {
    'max_seq_len': 32,
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 32,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda' #change this to run on cuda #'cuda:0'
}

In [7]:
#diagnos_codes = dataset['diagnos_code'].tolist()
#med_codes = dataset['medication_code'].tolist()
#diagnos_codes.extend(med_codes)
#file_path = '../data/vocabularies/Synthea/snomedrxnorm'
#write_codes_to_file(diagnos_codes, file_path)

In [8]:
#ages = dataset['age'].tolist()
#file_path = '../data/vocabularies/Synthea/age'
#write_age_to_file(ages, file_path)

In [9]:
files = {'code':'../data/vocabularies/Synthea/snomedrxnorm.npy',
         'age':'../data/vocabularies/Synthea/age.npy'
        }
tokenizer = EHRTokenizer(task='MLM', filenames=files)

In [10]:
#train, val, test = train_test_val_split(dataset, train_ratio=0.6, validation_ratio=0.2, test_ratio=0.2)
path='../data/datasets/synthea/'
#train.to_parquet(path + 'train')
#val.to_parquet(path + 'val')
#test.to_parquet(path + 'test')

train = pd.read_parquet(path + 'train')
val = pd.read_parquet(path + 'val')
test = pd.read_parquet(path + 'test')

In [11]:
condfiles = {'dd':'../data/train_stats/dd_cond_probs.empirical.p', 
             'dp':'../data/train_stats/dp_cond_probs.empirical.p', 
             'pp':'../data/train_stats/pp_cond_probs.empirical.p' , 
             'pd':'../data/train_stats/pd_cond_probs.empirical.p'
            }

In [12]:
traind = EHRDataset(train, max_len=train_params['max_len_seq'], conditional_files=condfiles, tokenizer=tokenizer)
vald = EHRDataset(val, max_len=train_params['max_len_seq'], tokenizer=tokenizer, conditional_files=condfiles, run_type='val')
testd = EHRDataset(test, max_len=train_params['max_len_seq'], tokenizer=tokenizer, conditional_files=condfiles, run_type='test')

Loading data
Loading data
Loading data


In [13]:
model_config = {
    'vocab_size': len(tokenizer.getVoc('code').keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, #tune.choice([100, 150, 288]), #288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(tokenizer.getVoc('age').keys()), # number of vocab for age embedding,
    'gender_vocab_size': 3,
    'max_position_embeddings': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 4, #4, # number of multi-head attention layers required
    'num_attention_heads': 6, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 300, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
    'use_prior':True,
}

In [14]:
tensorboarddir = '../logs/'

In [15]:
trainloader = torch.utils.data.DataLoader(traind, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(vald, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testd, batch_size=32, shuffle=True)

In [16]:
#from ray.tune.integration.pytorch_lightning import TuneReportCallback
#from functools import partial

In [17]:
pl.__version__

'1.5.10'

In [20]:
def train_test_model(model_config, epochs, tensorboarddir, save_model=False):
    
    '''
    callback = TuneReportCallback(
    {
        "loss": "val_loss",
        "mean_accuracy": "val_accuracy"
    },
    on="validation_step")
    '''
    
    trainer = pl.Trainer(
            max_epochs=epochs, 
            gpus=2,
            accelerator="ddp",
            #logger=pl.loggers.TensorBoardLogger(save_dir=tensorboarddir),
            callbacks=[pl.callbacks.TQDMProgressBar()], #progress.ProgressBar()], 
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10,
        )
    
    conf = BertConfig(model_config)
    model = BertForMaskedLM(conf) #BertForMaskedLM(conf)
    params = list(model.named_parameters())
    optim = adam(params, optim_param)
    
    patienttrajectory = TrainerMLM(model, optim, optim_param, 0.1, use_prior=model_config['use_prior'])
    print("Trainer is fitting")
    trainer.fit(
        patienttrajectory, 
        train_dataloader=trainloader,
        val_dataloaders=valloader,
    );
    print("Predicting on test data")
    predictions = trainer.predict(patienttrajectory, dataloaders=testloader)
    
    avg_acc = sum([ stats['precision'] for stats in predictions ]) / len(predictions)
    avg_acc*100
    
    #tune.report(mean_accuracy=avg_acc)
    print("Avg precision score: {}".format(avg_acc))
    
    if save_model:
        print("Saving model")
        PATH = '../saved_models/MLM/model_without_prior'
        torch.save(model.state_dict(), PATH)

In [21]:
train_test_model(model_config=model_config, epochs=5, tensorboarddir=tensorboarddir, save_model=False)

MisconfigurationException: `Trainer(strategy='ddp')` or `Trainer(accelerator='ddp')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible backends: dp, ddp_spawn, ddp_sharded_spawn, tpu_spawn. In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

### Visualize latent-space, namely the encoded space for each layer

In [17]:
model = BertForMaskedLM(conf) 
PATH = "../saved_models/MLM/model_with_prior_82test"
model = load_model(PATH, model)
params = list(model.named_parameters())
optim = adam(params, optim_param)

t_total value of -1 results in schedule not being applied


In [18]:
testsequence = next(iter(testloader))

In [21]:
attentionprobs = get_attention_scores(model, testsequence, 1, tokenizer, conf)